## Notebook para criação da base com as quantidades vendidas pelos valores mínimos, máximos e média por:
- SKU;
- Estado (Regional);

#### A ideia é dar uma visão para o lojista dos preços praticados nacionalmente e por região (estado)

##### Iniciando o Notebook Incluir no projeto o token para exposição na nuvem caso seja necessário

In [1]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(spark.sparkContext, 'de02faf2-fbe4-466f-b8cd-44296f096a9d', 'p-25957a2b4ca812c11c01435cf691ea3068358702')
pc = project.project_context

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20210530185816-0000
KERNEL_ID = dc93b153-965d-4cd9-a3be-237d55209b8f


**Importando algumas bibliotecas Utilizadas no projeto**

In [2]:
import sys
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

**Obter os dados e criar as tabelas temporárias para utilização com comandos SQL no spark**

In [3]:
dfcompraentregastatus = spark.read.parquet(project.get_file_url("compraentregastatus.parquet"))

In [4]:
dfcompraentrega = spark.read.parquet(project.get_file_url("compraentrega.snappy.parquet"))

In [5]:
# Criando uma tabela temporaria
dfcompraentrega.createOrReplaceTempView("cprEntrega")
dfcompraentregastatus.createOrReplaceTempView("cprEntregaStaus")

In [6]:
dfvariacao = spark.read.parquet(project.get_file_url("variacao_preco.parquet"))
dfvariacao.createOrReplaceTempView("variacao")

In [7]:
dfcprSku = spark.read.parquet(project.get_file_url("compraentregasku.snappy.parquet"))
dfcprSku.createOrReplaceTempView("compraSKU")

In [8]:
dfcompra = spark.read.parquet(project.get_file_url("compra.snappy.parquet"))
dfcompra.createOrReplaceTempView("compra")

**Junção das tabelas de compra juntamente com a tabela de variação de preço criado no outro notebook**
para obter os valores máximos, mínimo pratidos e o intervalo entre esses valores (valor entre)

In [9]:
spark.sql("SELECT tmp.IDSKUREFERENCIA, sum(qtde_min) as qtde_min, \
           sum(qtde_max) as qtde_max, \
           case when sum(total - qtde_min - qtde_max) < 0 then 0 \
            else sum(total - qtde_min - qtde_max) end as qtde_entre from (\
           select V.IDSKUREFERENCIA, 0 as qtde_min, 0 as qtde_max, count(*) as total \
             from compra     C \
                 ,cprEntrega E \
                 ,compraSKU  S \
                 ,variacao   V \
           where C.IDCOMPRA = E.IDCOMPRA \
             and E.IDCOMPRAENTREGA = S.IDCOMPRAENTREGA \
             and S.IDSKUREFERENCIA = V.IDSKUREFERENCIA \
             and cast(C.data as date) >= cast('2021-04-30' as date) - 60 \
             and E.idlojista in (10012,37450,92893,80333,64189)\
             group by V.IDSKUREFERENCIA\
          union all \
          select V.IDSKUREFERENCIA, count(*) as qtde_min, 0 as qtde_max, 0 as total \
            from compra     C \
                ,cprEntrega E \
                ,compraSKU  S \
                ,variacao   V \
           where C.IDCOMPRA = E.IDCOMPRA \
             and E.IDCOMPRAENTREGA = S.IDCOMPRAENTREGA \
             and S.IDSKUREFERENCIA = V.IDSKUREFERENCIA \
             and cast(C.data as date) >= cast('2021-04-30' as date) - 60 \
             and VALORVENDAUNIDADESEMDESCONTO = V.minVrVenda  \
             and E.idlojista in (10012,37450,92893,80333,64189)\
             group by V.IDSKUREFERENCIA\
             union all \
          select V.IDSKUREFERENCIA, 0 as qtde_min, count(*) as qtde_max, 0 as total \
            from compra     C \
                ,cprEntrega E \
                ,compraSKU  S \
                ,variacao   V \
           where C.IDCOMPRA = E.IDCOMPRA \
             and E.IDCOMPRAENTREGA = S.IDCOMPRAENTREGA \
             and S.IDSKUREFERENCIA = V.IDSKUREFERENCIA \
             and cast(C.data as date) >= cast('2021-04-30' as date) - 60 \
             and VALORVENDAUNIDADESEMDESCONTO = V.maxVrVenda  \
             and E.idlojista in (10012,37450,92893,80333,64189)\
             group by V.IDSKUREFERENCIA\
             ) as tmp \
             group by tmp.IDSKUREFERENCIA \
             ").show(truncate=False)

+---------------+--------+--------+----------+
|IDSKUREFERENCIA|qtde_min|qtde_max|qtde_entre|
+---------------+--------+--------+----------+
|8154506        |2       |2       |0         |
|1504540530     |1       |1       |0         |
|1504559122     |1       |1       |0         |
|1509144134     |1       |1       |0         |
|1504183454     |1       |1       |0         |
|1504540521     |1       |1       |0         |
|1501283149     |2       |2       |0         |
|1502998468     |1       |1       |0         |
|1502998701     |2       |2       |0         |
|1509146521     |1       |1       |0         |
|1512120618     |5       |5       |0         |
|1500125888     |1       |1       |0         |
|1504775619     |1       |1       |0         |
|1503675043     |1       |1       |0         |
|1503042197     |1       |1       |0         |
|13553255       |2       |1       |9         |
|1502998707     |2       |2       |0         |
|1501281810     |1       |1       |0         |
|1510076731  

**Tabela temporária** Cria a tabela gerada em dataframe para gravação em disco

In [10]:
dfTabelaFinal = spark.sql("SELECT tmp.IDSKUREFERENCIA, sum(qtde_min) as qtde_min, \
           sum(qtde_max) as qtde_max, \
           case when sum(total - qtde_min - qtde_max) < 0 then 0 \
            else sum(total - qtde_min - qtde_max) end as qtde_entre from (\
           select V.IDSKUREFERENCIA, 0 as qtde_min, 0 as qtde_max, count(*) as total \
             from compra     C \
                 ,cprEntrega E \
                 ,compraSKU  S \
                 ,variacao   V \
           where C.IDCOMPRA = E.IDCOMPRA \
             and E.IDCOMPRAENTREGA = S.IDCOMPRAENTREGA \
             and S.IDSKUREFERENCIA = V.IDSKUREFERENCIA \
             and cast(C.data as date) >= cast('2021-04-30' as date) - 60 \
             and E.idlojista in (10012,37450,92893,80333,64189)\
             group by V.IDSKUREFERENCIA\
          union all \
          select V.IDSKUREFERENCIA, count(*) as qtde_min, 0 as qtde_max, 0 as total \
            from compra     C \
                ,cprEntrega E \
                ,compraSKU  S \
                ,variacao   V \
           where C.IDCOMPRA = E.IDCOMPRA \
             and E.IDCOMPRAENTREGA = S.IDCOMPRAENTREGA \
             and S.IDSKUREFERENCIA = V.IDSKUREFERENCIA \
             and cast(C.data as date) >= cast('2021-04-30' as date) - 60 \
             and VALORVENDAUNIDADESEMDESCONTO = V.minVrVenda  \
             and E.idlojista in (10012,37450,92893,80333,64189)\
             group by V.IDSKUREFERENCIA\
             union all \
          select V.IDSKUREFERENCIA, 0 as qtde_min, count(*) as qtde_max, 0 as total \
            from compra     C \
                ,cprEntrega E \
                ,compraSKU  S \
                ,variacao   V \
           where C.IDCOMPRA = E.IDCOMPRA \
             and E.IDCOMPRAENTREGA = S.IDCOMPRAENTREGA \
             and S.IDSKUREFERENCIA = V.IDSKUREFERENCIA \
             and cast(C.data as date) >= cast('2021-04-30' as date) - 60 \
             and VALORVENDAUNIDADESEMDESCONTO = V.maxVrVenda  \
             and E.idlojista in (10012,37450,92893,80333,64189)\
             group by V.IDSKUREFERENCIA\
             ) as tmp \
             group by tmp.IDSKUREFERENCIA \
             ")

**GRAVAÇÃO EM DISCO** Os próximos passos é para gravação da tabela gerada em disco e disponibilizar nos ativos

In [14]:
dfTabelaFinal.repartition(1).write.mode("overwrite").parquet("/home/spark/shared/tmp/compra_produto.parquet")

In [12]:
dfTabelaFinal.repartition(1).write.mode("overwrite").csv('/home/spark/shared/tmp/variacao_preco_qtde.csv')

In [50]:
# Obtendo credenciais para disponibilizar nos ativos
credentials_1 = {
    'IAM_SERVICE_ID': 'iam-ServiceId-f8ddc357-a261-42a2-a6f2-44ac38d6cc50',
    'IBM_API_KEY_ID': '5y97tXSIMk-o11IIlG-qhORjxg4zQ6yFprxqB54ntyX9',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.cloud.ibm.com/oidc/token',
    'BUCKET': 'projetoviagrupo6-donotdelete-pr-ehmgbn0j3d7dfc',
}

cgsClient = ibm_boto3.client(service_name='s3',
    ibm_api_key_id = credentials_1['IBM_API_KEY_ID'],
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

In [51]:
ls tmp/variacao_preco_qtde.parquet/

part-00000-f8ebc172-c6f1-4300-b5b8-52447749d310-c000.snappy.parquet  _SUCCESS


In [57]:
cgsClient.upload_file(Filename='tmp/variacao_preco_qtde.parquet/part-00000-f8ebc172-c6f1-4300-b5b8-52447749d310-c000.snappy.parquet',Bucket=credentials_1['BUCKET'],Key='variacao_preco_qtde.parquet')

In [54]:
ls tmp/variacao_preco_qtde.csv/

part-00000-b9bf1c06-36b0-4d05-8dff-0cbf27aa2c22-c000.csv  _SUCCESS


In [56]:
cgsClient.upload_file(Filename='tmp/variacao_preco_qtde.csv/part-00000-b9bf1c06-36b0-4d05-8dff-0cbf27aa2c22-c000.csv',Bucket=credentials_1['BUCKET'],Key='variacao_preco_qtde.csv')